# Loan Prediction

## 0. Import Packages

In [1]:
import pandas as pd
import numpy as np

## 1. Prepare Data

In [2]:
loan_train = pd.read_csv("train_ctrUa4K.csv")
loan_train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [3]:
# count the null columns
null_columns = loan_train.columns[loan_train.isnull().any()]
loan_train[null_columns].isnull().sum()

Gender              13
Married              3
Dependents          15
Self_Employed       32
LoanAmount          22
Loan_Amount_Term    14
Credit_History      50
dtype: int64

## 2. Feature Engineering

In [4]:
# drop null values
loan_train = loan_train.dropna()

In [5]:
X = loan_train.iloc[:, 1:-1]
y = loan_train.iloc[:, -1]

In [6]:
print(X.shape)
print(y.shape)

(480, 11)
(480,)


### 2.1 One Hot Encoding to Certain Columns

In [7]:
# these four columns need to be onehotted
onehot_targets = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 
                  'Property_Area']

In [8]:
for target in onehot_targets:
    onehot_temp = pd.get_dummies(X[target])
    X = X.drop(target, axis=1)
    X = pd.concat([onehot_temp, X], axis=1)

### 2.2 Label Encoding to Predicted Value

In [10]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)

LabelEncoder()

In [11]:
y = le.transform(y)

In [14]:
# prepare train and validation data
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## 3. Prepare Model

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
clf = RandomForestClassifier()

## 4. Train Model

In [17]:
clf.fit(X_train, y_train)

/home/ccc/anaconda3/envs/mlnote/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

## 5 Evaluate Model

In [18]:
print('accuracy: ', clf.score(X_val, y_val))

accuracy:  0.7708333333333334
